In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from PIL import Image

In [2]:
fs=4000

In [3]:
import glob
path = '../input/emg2channels'
all_files = glob.glob(path+'/EMG-S*/'+'*.csv')
dflist=[]
for ii in range(len(all_files)):
    dflist.append(pd.read_csv(all_files[ii],header=None))

In [4]:
len(all_files)

600

In [5]:
label=['HC','II','LL','MM','RR','TI','TL','TM','TR','TT']
datanumber=[0,0,0,0,0,0,0,0,0,0]

In [6]:
infodf = pd.DataFrame(columns=['filename', 'class'])

In [7]:
!mkdir /kaggle/working/data
!mkdir /kaggle/working/data/numpy
!mkdir /kaggle/working/data/image

In [8]:
max = 60
for ii in range(len(dflist)):
#     print(ii,all_files[ii])
    file=''.join(''.join(all_files[ii].split('/')[3:5]).split('.')[0].split('-')[1:3])
    df=dflist[ii]
    df0=np.array(df[0])
    df1=np.array(df[1])
    N=len(df)
    t=np.array(range(N))/fs

    from scipy.fft import fft, fftfreq
    import matplotlib.pyplot as plt
    from scipy import signal
#     print(file)
    for ii in range(49):
        if len(file)==6:
            labelpos=label.index(file[3:5])
            datanumber[labelpos]=datanumber[labelpos]+1
            fn=int(file[1:3])
        else:
            labelpos=label.index(file[2:4])
            datanumber[labelpos]=datanumber[labelpos]+1
            fn=int(file[1])
            
        f, t, Zxx = signal.stft(df0[ii*fs//10:(ii+2)*fs//10], fs, boundary = None, nperseg=int(fs/10))
        data0=20*np.log10(np.abs(Zxx))
        f, t, Zxx = signal.stft(df1[ii*fs//10:(ii+2)*fs//10], fs, boundary = None, nperseg=int(fs/10))
        data1=20*np.log10(np.abs(Zxx))
        data01=np.concatenate(((data0[0:60,:],data1[0:60,:])),axis=1)
        datamin=data01.min()
        datamax=data01.max()
        data01 = data01-datamax
        data01 = np.where(data01<-max, -max,data01)
        data01 = np.uint8((data01+max)*255//max)
        
        setnum=str(fn+10)
        datanum=str(10000+datanumber[labelpos])
        labelnum=str(labelpos+11)
#         print(ii)
        path = '/kaggle/working/data/numpy/'
        np.save(path+setnum+datanum+labelnum,data01)
        path = '/kaggle/working/data/image/'
        Image.fromarray(data01).save(path+setnum+datanum+labelnum+'.png')
        infodf = infodf.append({'filename': setnum+datanum+labelnum, 'class': int(labelnum)-11},ignore_index=True)
#         print(infodf)
    infodf.to_csv('info.csv')

In [9]:
import shutil
shutil.move('info.csv',"/kaggle/working/data")

'/kaggle/working/data/info.csv'

In [10]:
!zip -r -q output.zip data/
!rm -rf  data/*